# Генерация новых признаков

In [1]:
import pandas as pd
import numpy as np

In [2]:
# загружаем данные
clients_df = pd.read_csv('datasets/clients.csv', dtype={'employee_count_nm': 'category', 'bankemplstatus': 'bool', 'customer_age': 'category'})
train_df = pd.read_csv('datasets/train.csv', dtype={'target': 'bool'})
reports_df = pd.read_csv('datasets/report_dates.csv', parse_dates=['report_dt'])
transactions_df = pd.read_csv('datasets/transactions.csv.zip', compression='zip', 
                              dtype={'mcc_code': 'category', 'currency_rk': 'category'}, parse_dates=['transaction_dttm'])

In [3]:
# объединим два поддиапазона в один
clients_df.loc[clients_df.employee_count_nm.isin(['ОТ 11 ДО 30', 'ОТ 31 ДО 50']), 'employee_count_nm'] = 'ОТ 11 ДО 50'

# пока что определим "БОЛЕЕ 500" как "ОТ 501 ДО 1000"
clients_df.loc[clients_df.employee_count_nm == 'БОЛЕЕ 500', 'employee_count_nm'] = 'ОТ 501 ДО 1000'

# заполним пропуски как "unknown"
clients_df['employee_count_nm'] = clients_df['employee_count_nm'].cat.add_categories('unknown')
clients_df['employee_count_nm'] = clients_df['employee_count_nm'].fillna('unknown')

# удалим лишние категории
clients_df['employee_count_nm'] = clients_df['employee_count_nm'].cat.remove_unused_categories()

# проранжируем категории
clients_df['employee_count_nm'] = clients_df['employee_count_nm'].cat.reorder_categories(
    ['ДО 10', 'ОТ 11 ДО 50', 'ОТ 51 ДО 100', 'ОТ 101 ДО 500', 'ОТ 501 ДО 1000', 'БОЛЕЕ 1001', 'unknown'], ordered=True)

In [4]:
transactions_df['tran_date'] = transactions_df['transaction_dttm'].dt.date              # дата транзакции
transactions_df['tran_day_of_week'] = transactions_df['transaction_dttm'].dt.dayofweek  # день недели, в который была произведена транзакция
transactions_df['tran_month'] = transactions_df['transaction_dttm'].dt.month            # месяц, в котором была произведена транзакция

# была ли транзакция в рабочее время с 8:00 до 18:00 в рабочие дни
transactions_df['is_tran_in_working_time'] = (transactions_df['transaction_dttm'].dt.hour >= 8) & (transactions_df['transaction_dttm'].dt.hour < 18) &\
                                             (transactions_df['tran_day_of_week'].isin([5, 6]))

# была ли транзакция в выходные
transactions_df['is_tran_in_weekend'] = transactions_df['tran_day_of_week'].isin([5, 6])

transactions_df = transactions_df.merge(clients_df, on='user_id', how='left')
transactions_df = transactions_df.merge(reports_df, on='report', how='left')

 # количество дней от транзакции до отчета
transactions_df['days_from_tran_to_report'] = (transactions_df['report_dt'] - transactions_df['transaction_dttm']).dt.days

In [5]:
top_15_mcc = transactions_df.mcc_code.value_counts(normalize=True).head(15)

currency_rk_freq = transactions_df.currency_rk.value_counts(normalize=True)
currency_rk_most_freq = currency_rk_freq[currency_rk_freq > 0.01]

In [9]:
def generate_features_by_tendencies(transactions_df, start_name: str) -> pd.DataFrame:
    day_of_week = {
        0: 'monday',
        1: 'tuesday',
        2: 'wednesday',
        3: 'thursday',
        4: 'friday',
        5: 'saturday',
        6: 'sunday',
    }
    
    tendencies = ['sum', 'mean', 'median', 'max', 'min', 'std', 'count']
    
    # тенденции по всем транзакциям за всё время
    full_df = transactions_df.groupby('user_id').agg({'transaction_amt': tendencies}).reset_index()
    full_df.columns = ['user_id'] + [f'{start_name}_tran_{tendency}' for tendency in tendencies]
        
    # тенденции по всем транзакциям за день
    df = transactions_df.groupby(['user_id', 'tran_date']).agg({'transaction_amt': ['sum']}).reset_index()
    df.columns = ['user_id', 'tran_date', 'tran_sum_by_day']
    df = df.drop('tran_date', axis=1)
    df = df.groupby('user_id').agg({'tran_sum_by_day': tendencies[1:-1]}).reset_index()
    df.columns = ['user_id'] + [f'{start_name}_{col}_{tendency}' for col, tendency in df.columns[1:]]
    full_df = full_df.merge(df, on='user_id', how='left')     
    
    # тенденции по всем транзакциям за месяц
    df = transactions_df.groupby(['user_id', 'tran_month']).agg({'transaction_amt': ['sum']}).reset_index()
    df.columns = ['user_id', 'tran_month', 'tran_sum_by_month']
    df = df.drop('tran_month', axis=1)
    df = df.groupby('user_id').agg({'tran_sum_by_month': tendencies[1:-1]}).reset_index()
    df.columns = ['user_id'] + [f'{start_name}_{col}_{tendency}' for col, tendency in df.columns[1:]]
    full_df = full_df.merge(df, on='user_id', how='left')
    
    # тенденции по всем транзакциям по дням недели
    df = transactions_df.pivot_table(index='user_id', columns='tran_day_of_week', values='transaction_amt', aggfunc=tendencies).fillna(0).reset_index()
    df.columns = ['user_id'] + [f'{start_name}_tran_{day_of_week[day]}_{tendency}' for tendency, day in df.columns[1:]]
    
    # число уникальных MCC кодов
    full_df = full_df.merge(transactions_df.groupby('user_id')['mcc_code'].nunique().reset_index(name=f'{start_name}_mcc_nunique'), on='user_id', how='left')
    
    # тенденции по топ-15 mcc
    df = transactions_df.query('mcc_code in @top_15_mcc.index.values').pivot_table(index='user_id', values='transaction_amt', columns='mcc_code', 
                                                                                   aggfunc=tendencies, observed=True).fillna(0).reset_index()
    df.columns = ['user_id'] + [f'{start_name}_mcc_{mcc}_{tendency}' for tendency, mcc in df.columns[1:]]
    full_df = full_df.merge(df, on='user_id', how='left')
    
    # тенденции по всем транзакциям по виду валюты
    df = transactions_df.query('currency_rk in @currency_rk_most_freq.index').pivot_table(index='user_id', columns='currency_rk', values='transaction_amt', 
                                                                                     aggfunc='count', observed=True).reset_index()
    df.columns = ['user_id'] + [f'{start_name}_currency_{col}_count' for col in df.columns[1:]]
    full_df = full_df.merge(df, on='user_id', how='left')
    
    full_df = full_df.fillna(0)
    
    # число уникальных дней и месяцев транзакций
    full_df = full_df.merge(transactions_df.groupby('user_id')['tran_date'].nunique().reset_index(name=f'{start_name}_tran_date_nunique'), on='user_id', how='left')
    full_df = full_df.merge(transactions_df.groupby('user_id')['tran_month'].nunique().reset_index(name=f'{start_name}_tran_month_nunique'), on='user_id', how='left')
    
    # тенденции по транзакциям в рабочее время
    df = transactions_df.pivot_table(index='user_id', columns='is_tran_in_working_time', values='transaction_amt', aggfunc=tendencies).fillna(0).reset_index()
    df.columns = ['user_id'] + [f'{start_name}_tran_in_working_time_({is_true})_{tendency}' for tendency, is_true in df.columns[1:]]
    full_df = full_df.merge(df, on='user_id', how='left')
    
    # тенденции по транзакциям в выходные
    df = transactions_df.pivot_table(index='user_id', columns='is_tran_in_weekend', values='transaction_amt', aggfunc=tendencies).fillna(0).reset_index()
    df.columns = ['user_id'] + [f'{start_name}_tran_in_weekend_({is_true})_{tendency}' for tendency, is_true in df.columns[1:]]
    full_df = full_df.merge(df, on='user_id', how='left')
    
    # тенденции по времени до отчета
    df = transactions_df.groupby('user_id').agg({'days_from_tran_to_report': tendencies[1:-1]}).reset_index()
    df.columns = ['user_id'] + [f'{start_name}_days_to_report_{tendency}' for _, tendency in df.columns[1:]]
    full_df = full_df.merge(df, on='user_id', how='left')    
    
    return full_df

In [10]:
def generate_features(transactions, clients, train) -> pd.DataFrame:
    full_df = clients[['user_id']]
    
    for tran_type in ['total', 'pos', 'neg']:
        print(f'Generating features by {tran_type} transactions ...')
        
        if tran_type == 'total':
            transactions_df = transactions.copy()
        elif tran_type == 'pos':
            transactions_df = transactions[transactions['transaction_amt'] > 0].copy()
        elif tran_type == 'neg':
            transactions_df = transactions[transactions['transaction_amt'] < 0].copy()
            
        full_df = full_df.merge(generate_features_by_tendencies(transactions_df, tran_type), on='user_id', how='left')
        
    full_df = clients_df.merge(full_df, on='user_id', how='left')
    
    num_cols = [col for col in full_df.columns if col not in ['user_id', 'report', 'employee_count_nm', 'bankemplstatus', 'customer_age']]
    for col in num_cols:
        full_df[col] = pd.to_numeric(full_df[col])
        
    full_df[num_cols] = full_df[num_cols].fillna(0)
    full_df = full_df.merge(train, on='user_id', how='left')
        
    return full_df

In [11]:
full_df = generate_features(transactions_df, clients_df, train_df)

Generating features by total transactions ...
Generating features by pos transactions ...
Generating features by neg transactions ...


In [14]:
full_df.to_csv('datasets/preprocessed_v0.1.csv', index=False, compression='zip')

In [15]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96000 entries, 0 to 95999
Columns: 487 entries, user_id to time
dtypes: bool(1), category(2), float64(475), int64(8), object(1)
memory usage: 354.8+ MB
